In [1]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

def discretize_value(value, col, global_min, global_max, num_bins=20):
    """
    Дискретизирует числовое значение признака в один из num_bins бинов.
    Если value или глобальные min/max равны NaN, возвращается 'bin0'.
    """
    minimum = global_min[col]
    maximum = global_max[col]
    
    if maximum == minimum:
        return "bin0"
    
    bin_index = int((value - minimum) / (maximum - minimum) * num_bins)
    if bin_index >= num_bins:
        bin_index = num_bins - 1
    return f"bin{bin_index}"

def discretize_range(min_value, max_value, col, global_min, global_max, num_bins=20):
    """
    Для диапазона значений: дискретизирует минимальное и максимальное значение признака и,
    если они различаются, объединяет в строку вида 'binX-binY'.
    Если же оба значения попадают в один бин, возвращает просто его.
    """
    min_bin = discretize_value(min_value, col, global_min, global_max, num_bins)
    max_bin = discretize_value(max_value, col, global_min, global_max, num_bins)
    return min_bin if min_bin == max_bin else f"{min_bin}-{max_bin}"

def main():
    # Имя файла с датасетом
    input_file = r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\ML&TEST\merged_sampled.csv'
    
    # Задаем списки столбцов:
    # numeric_features – числовые признаки, для которых будем определять диапазон.
    numeric_features = ['Flow Duration', 'Active Mean', 'Idle Mean', 'Average Packet Size',
                          'Total Fwd Packets', 'Total Length of Fwd Packets'] #'Flow Packets/s'
    # raw_features – столбцы, для которых не требуется дискретизация,
    # для них выбирается наиболее частое значение.
    raw_features = ['FIN Flag Count', 'SYN Flag Count', 
                    'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Destination Port']
    
    # Чтение датасета
    df = pd.read_csv(input_file)
    
    # Проверяем наличие столбца Label
    if 'Label' not in df.columns:
        raise ValueError("Датасет должен содержать столбец 'Label' с типами атак или поведением.")
    
    # Фильтруем датасет: исключаем benign (обычное поведение)
    df_attacks = df[df['Label'] != 'BENIGN']
    
    # Отбираем только те столбцы, которые присутствуют в датасете
    numeric_features = [col for col in numeric_features if col in df_attacks.columns]
    raw_features = [col for col in raw_features if col in df_attacks.columns]
    
    # Вычисляем глобальные минимумы и максимумы для числовых признаков
    global_min = df_attacks[numeric_features].min()
    global_max = df_attacks[numeric_features].max()
    
    # Словарь для хранения шаблонов атак (один шаблон на тип атаки)
    templates = {}
    
    # Группируем весь датасет по столбцу Label
    grouped = df_attacks.groupby('Label')
    for label, group in grouped:
        template = {}
        # Для каждого числового признака вычисляем диапазон (min и max) и дискретизируем их
        group_mins = group[numeric_features].min()
        group_maxs = group[numeric_features].max()
        for col in numeric_features:
            min_val = group_mins[col]
            max_val = group_maxs[col]
            template[col] = discretize_range(min_val, max_val, col, global_min, global_max, num_bins=20)
        
        # Для "сырых" признаков выбирается наиболее часто встречающееся значение (мода)
        for col in raw_features:
            if group[col].dropna().empty:
                template[col] = "N/A"
            else:
                mode_val = group[col].mode()
                template[col] = mode_val.iloc[0] if not mode_val.empty else "N/A"
        
        templates[label] = template
    
    # Запись шаблонов атак в файл
    output_file = r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\result\sample\attack_templates_range3.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        for attack, pattern in templates.items():
            f.write(f"{attack}{{\n")
            for col, val in pattern.items():
                f.write(f"  {col}: {val}\n")
            f.write("}\n\n")
    
    print(f"Шаблоны для всех найденных паттернов атак сохранены в файл {output_file}")

if __name__ == '__main__':
    main()


Шаблоны для всех найденных паттернов атак сохранены в файл C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\result\sample\attack_templates_range3.txt
